In [1]:
import cv2
import numpy as np
from PIL import Image
import pandas
import math
from generate_inputs import TrajectoryDataset
from generate_inputs import device
from generate_inputs import *
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader

In [2]:
train_dataset = TrajectoryDataset('./data/', np.arange(2,3) , 50, 0.5, math.radians(10))
print(len(train_dataset))
train_dataset[0]['image2']

2
13632


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [3]:
# Verifying input dimensions

device = torch.device("cuda")
net = Net()
net = net.to(device)
print(net)

input = torch.randn(1, 2, 224, 224).to(device)
out = net(input)
print(out.shape)

x = np.stack([train_dataset[0]['image1'], train_dataset[0]['image2']])
x = np.expand_dims(x, axis=0)
print(x.shape)

Net(
  (conv_layers): Sequential(
    (0): Conv2d(2, 16, kernel_size=(5, 5), stride=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(16, 32, kernel_size=(5, 5), stride=(2, 2))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(32, 64, kernel_size=(5, 5), stride=(2, 2))
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2))
    (10): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): Conv2d(128, 256, kernel_size=(5, 5), stride=(2, 2))
    (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=4096, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_feat

In [ ]:
# Training X- network

import random
import torch.optim as optim
idxs = list(range(len(train_dataset)))
random.shuffle(idxs)
net_x = Net()
net_x = net_x.to(device)
optimizer = optim.Adam(net_x.parameters(), lr=0.0001)
train_idxs = idxs[:int(len(train_dataset) * 0.8)]
test_idxs = idxs[int(len(train_dataset) * 0.8):]
min_loss = 1
for epoch in range(10):
    train_total = 0
    test_total = 0
    for idx in test_idxs:
        x = np.stack([train_dataset[idx]['image1'], train_dataset[idx]['image2']])
        x = torch.tensor(np.expand_dims(x, axis=0)).float().to(device)
        out = net_x(x)
        target = torch.tensor(train_dataset[idx]['rel_pose'][0]).float().to(device)
        #criterion = nn.MSELoss()
        criterion = nn.L1Loss()
        loss = criterion(out, target)
        test_total += loss.item()
    print("Test loss:", test_total / len(test_idxs))
    if min_loss > test_total / len(test_idxs):
        print("New lowest test loss, saved!")
        torch.save(net_x, "net_x_3.pyt")
        min_loss = test_total / len(test_idxs)
    
    for idx in train_idxs:
        x = np.stack([train_dataset[idx]['image1'], train_dataset[idx]['image2']])
        x = torch.tensor(np.expand_dims(x, axis=0)).float().to(device)
        out = net_x(x)
        target = torch.tensor(train_dataset[idx]['rel_pose'][0]).float().to(device)
        #print(out.shape, target.shape)
        #criterion = nn.MSELoss()
        criterion = nn.L1Loss()
        loss = criterion(out, target)
        optimizer.zero_grad()
        loss.backward()
        train_total += loss.item()
        optimizer.step()     
    print("Train loss:", train_total / len(train_idxs))


Test loss: 0.2830528800898625
New lowest test loss, saved!
Train loss: 0.05234691195663087
Test loss: 0.024577219203130902
New lowest test loss, saved!
Train loss: 0.02204110767420745
Test loss: 0.016972422945751325
New lowest test loss, saved!
Train loss: 0.016967406287850834
Test loss: 0.015579401887680066
New lowest test loss, saved!
Train loss: 0.014532741579897137
Test loss: 0.012835956803585503
New lowest test loss, saved!
Train loss: 0.012878421722223376
Test loss: 0.0161179176664583
Train loss: 0.011705440559101209
Test loss: 0.010510227922316016
New lowest test loss, saved!
Train loss: 0.010702822007262778


In [ ]:
# Training Y- network

import random
import torch.optim as optim
idxs = list(range(len(train_dataset)))
random.shuffle(idxs)
net_y = Net()
net_y = net_y.to(device)
optimizer = optim.Adam(net_y.parameters(), lr=0.0001)
train_idxs = idxs[:int(len(train_dataset) * 0.8)]
test_idxs = idxs[int(len(train_dataset) * 0.8):]
min_loss = 1
for epoch in range(10):
    train_total = 0
    test_total = 0
    for idx in test_idxs:
        x = np.stack([train_dataset[idx]['image1'], train_dataset[idx]['image2']])
        x = torch.tensor(np.expand_dims(x, axis=0)).float().cuda()
        out = net_y(x)
        target = torch.tensor(train_dataset[idx]['rel_pose'][1]).float().to(device)
        #criterion = nn.MSELoss()
        criterion = nn.L1Loss()
        loss = criterion(out, target)
        test_total += loss.item()
    print("Test loss:", test_total / len(test_idxs))
    if min_loss > test_total / len(test_idxs):
        print("New lowest test loss, saved!")
        torch.save(net_y, "net_y_3.pyt")
        min_loss = test_total / len(test_idxs)
    
    for idx in train_idxs:
        x = np.stack([train_dataset[idx]['image1'], train_dataset[idx]['image2']])
        x = torch.tensor(np.expand_dims(x, axis=0)).float().cuda()
        out = net_y(x)
        target = torch.tensor(train_dataset[idx]['rel_pose'][1]).float().to(device)
        #criterion = nn.MSELoss()
        optimizer.zero_grad()
        criterion = nn.L1Loss()
        loss = criterion(out, target)
        loss.backward()
        train_total += loss.item()
        optimizer.step()     
    print("Train loss:", train_total / len(train_idxs))


In [ ]:
# Training Angle- network

import random
import torch.optim as optim
idxs = list(range(len(train_dataset)))
random.shuffle(idxs)
net_angle = Net()
net_angle = net_angle.to(device)
optimizer = optim.Adam(net_angle.parameters(), lr=0.0001)
train_idxs = idxs[:int(len(train_dataset) * 0.8)]
test_idxs = idxs[int(len(train_dataset) * 0.8):]
min_loss = 1
for epoch in range(10):
    train_total = 0
    test_total = 0
    for idx in test_idxs:
        x = np.stack([train_dataset[idx]['image1'], train_dataset[idx]['image2']])
        x = torch.tensor(np.expand_dims(x, axis=0)).float().cuda()
        out = net_angle(x)
        target = torch.tensor(train_dataset[idx]['rel_pose'][2]).float().to(device)
        criterion = nn.L1Loss()
        loss = criterion(out, target)
        test_total += loss.item()
    print("Test loss:", test_total / len(test_idxs))
    if min_loss > test_total / len(test_idxs):
        print("New lowest test loss, saved!")
        torch.save(net_angle, "net_angle_3.pyt")
        min_loss = test_total / len(test_idxs)
    
    for idx in train_idxs:
        x = np.stack([train_dataset[idx]['image1'], train_dataset[idx]['image2']])
        x = torch.tensor(np.expand_dims(x, axis=0)).float().cuda()
        out = net_angle(x)
        target = torch.tensor(train_dataset[idx]['rel_pose'][2]).float().to(device)
        optimizer.zero_grad()
        criterion = nn.L1Loss()
        loss = criterion(out, target)
        loss.backward()
        train_total += loss.item()
        optimizer.step()     
    print("Train loss:", train_total / len(train_idxs))
